In [2]:
import torch
import torch.nn as nn
import lightning as pl
from torchmetrics import Accuracy, F1Score

from src.data.dataset import SensorDataModule
from src.data.partition_helper import get_partitioned_data, get_partition_paths

dataset = SensorDataModule(get_partition_paths("./data/splits", k_folds=5), batch_size=32)

dataset.setup()

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchmetrics import Accuracy, F1Score

class BidirectionalLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout)

    def forward(self, x):
        x, _ = self.lstm(x)
        return x

class ForwardLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        x, _ = self.lstm(x)
        return x

class CustomBidirectionalLayer(nn.Module):
    def __init__(self, forward_layer, input_dim, hidden_dim, num_layers, dropout):
        super().__init__()
        self.forward_layer = forward_layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        
    def init_weights_from_forward_layer(self):
        # Fetch the parameters of the forward LSTM layer from ForwardLayer
        forward_params = list(self.forward_layer.lstm.parameters())
        # Fetch the parameters of the forward part of the bidirectional LSTM
        bidirectional_params = list(self.lstm.parameters())

        # Half the parameters belong to the forward LSTM part of the bidirectional LSTM (assuming the forward and backward have the same number of parameters)
        for i in range(len(forward_params)):
            if i < len(bidirectional_params) // 2:
                bidirectional_params[i].data.copy_(forward_params[i].data)

    def forward(self, x):
        # Ensure the forward weights are updated each time forward is called
        self.init_weights_from_forward_layer()
        x, _ = self.lstm(x)
        return x

class ResidualLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_blocks):
        super().__init__()
        self.bidirectional_layers = nn.ModuleList([
            BidirectionalLayer(input_dim if i == 0 else hidden_dim * 2, hidden_dim, num_layers, dropout)
            for i in range(num_blocks)
        ])

        output_dim_from_lstm = hidden_dim * 2

        self.fc_layers = nn.ModuleList([
            nn.Linear(input_dim if i == 0 else output_dim_from_lstm, output_dim_from_lstm)
            for i in range(num_blocks + 2)
        ])

        self.batch_norm = nn.BatchNorm1d(output_dim_from_lstm)

    def forward(self, x):

        x = self.bidirectional_layers[0](x)
        residual = F.relu(self.fc_layers[1](x))

        x = self.bidirectional_layers[1](residual)
        x = F.relu(self.fc_layers[2](x)) + residual  

        x = x.transpose(1, 2)  
        x = self.batch_norm(x)
        x = x.transpose(1, 2)  

        x = F.relu(self.fc_layers[3](x))  

        return x

class DeepBidirectionalLSTMs(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_blocks, output_dim):
        super().__init__()
        self.save_hyperparameters()
        self.lstm_networks = nn.ModuleList([ResidualLayer(hidden_dim if i == 0 else hidden_dim * 2, hidden_dim, num_layers, dropout, num_blocks) for i in range(num_blocks)])
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim * 2, output_dim)
        self.accuracy = Accuracy(task='multiclass', num_classes=output_dim)
        self.f1_score = F1Score(num_classes=output_dim, average='weighted', task='multiclass')

    def forward(self, x):
        # x initially has shape [batch_size, seq_length, input_dim], e.g., [32, 251, 64]
        batch_size, seq_len, _ = x.size()

        # Flatten the batch and sequence length dimensions to apply the FC layer to each timestep individually
        x = x.view(-1, x.size(2))  # Reshape to [batch_size * seq_length, input_dim], e.g., [32*251, 64]
        x = F.relu(self.fc1(x))    # Transform each timestep through the FC layer, output shape [32*251, hidden_dim]

        # Restore the original sequence format
        x = x.view(batch_size, seq_len, -1)  # Reshape back to [batch_size, seq_length, hidden_dim]

        # Process each sequence through the LSTM layers
        for lstm in self.lstm_networks:
            x = lstm(x)

        # Take the last timestep's output for classification
        x = x[:, -1, :]  # Output of the last timestep, shape [batch_size, hidden_dim]
        x = self.fc2(x)  # Final classification layer, output shape [batch_size, output_dim]
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y.float())
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', self.accuracy(logits, y))
        self.log('train_f1', self.f1_score(logits, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y.float())
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', self.accuracy(logits, y), prog_bar=True)
        self.log('val_f1', self.f1_score(logits, y), prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)



In [4]:

for key, fold in dataset.data_dict.items():
    train_dataloader, val_dataloader = fold['train'], fold['validate']
    trainer = pl.Trainer(max_epochs=5, devices=1, accelerator='mps', log_every_n_steps=10)
    model = DeepBidirectionalLSTMs(input_dim=48, hidden_dim=64, output_dim=dataset.num_classes, num_layers=1, dropout=0.2, num_blocks=2)
    trainer.fit(model, train_dataloader, val_dataloader)
    break


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/dmnk/PycharmProjects/cdl1-sensor-based/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/dmnk/PycharmProjects/cdl1-sensor-based/.venv/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warn

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
/Users/dmnk/PycharmProjects/cdl1-sensor-based/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [8]:
# Assuming `train_dataloader` is already defined and available
# Fetch the first batch
first_batch = next(iter(dataset.data_dict[0]['validate']))

# Unpack the first batch
data_tensors, labels = first_batch

# Print shapes and types to understand the structure
print("Data tensors shape:", data_tensors.shape)
print("Data tensors type:", type(data_tensors))
print("Labels shape:", labels.shape)
print("Labels type:", type(labels))

# If your tensors are dictionaries (which might be causing the error), print the keys
if isinstance(data_tensors, dict):
    print("Data tensor keys:", data_tensors.keys())

# Optionally, you can visualize or print part of the tensors to understand the actual data
print("First few data points:", data_tensors[:5])  # Adjust slicing based on your data size
print("First few labels:", labels[:5])


Data tensors shape: torch.Size([32, 251, 48])
Data tensors type: <class 'torch.Tensor'>
Labels shape: torch.Size([32, 5])
Labels type: <class 'torch.Tensor'>
First few data points: tensor([[[ 1.5077e-02, -8.4308e-01, -1.2617e+00,  ..., -6.7758e-01,
          -5.3039e-02,  5.5554e-01],
         [ 8.8432e-03, -8.4308e-01, -1.2617e+00,  ..., -6.7881e-01,
          -5.3039e-02,  5.5554e-01],
         [ 1.1694e-02, -8.4308e-01, -1.2617e+00,  ..., -6.7948e-01,
          -5.3039e-02,  5.5554e-01],
         ...,
         [-9.1835e-02, -8.4308e-01, -1.2617e+00,  ...,  8.5037e-01,
          -5.3039e-02,  5.5554e-01],
         [ 2.3238e-01, -8.4308e-01, -1.2617e+00,  ...,  8.3910e-01,
          -5.3039e-02,  5.5554e-01],
         [ 3.1028e-01, -8.4308e-01, -1.2617e+00,  ...,  8.4028e-01,
          -5.3039e-02,  5.5554e-01]],

        [[-9.4379e-04, -8.4308e-01,  9.5124e-01,  ..., -9.8356e-01,
           7.5225e-01,  3.7390e-01],
         [-1.0288e-03, -8.4308e-01,  9.5124e-01,  ..., -9.7817e-01,
